DATA CLEANING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = '/Users/banna/Desktop/CEE 4803/HW/HW2/1989-2019 FDOT Pavement Data.csv'
df = pd.read_csv(path)

Rows with the following stand to be deleted.
    
> A single empty value within their repair cycle designated on that row OR

> Any instance of a later quality rating greater than a previous quality rating within the repair cycle OR

> Any instance of a quality rating of 0 OR

> Any instance of a zero or NaN in any row of the predictor columns.
    
> Any instance of a duplicate row.

CLEAN CRACK DATA WITH LOCATION AND PREDICTOR DATA IN THE LEFT COLUMNS

In [ ]:
# Define columns to drop
columns_to_drop = ['DISTRICT', 'RCILM', 'SYSTEM', 'TYPE', 'SURFTYPE', 'DEFREAS']
df = df.drop(columns=columns_to_drop)

# Define columns of interest
crk_years = [str(year) for year in range(1989, 2020)]
crk_columns = [f'CRK{year}' for year in crk_years]

# Extract columns related to ratings and predictors
df_id = df.loc[:, 'RDWYID':'EMP']
df_predictors = df.loc[:, 'MAXSPEED':'FC']
df_cycle = df.loc[:, 'cyclastyr':'cycdefage']

df_ratings = df.loc[:, crk_columns]
df_rut = df.loc[:, 'RUT1989':'RUT2019']
df_ride = df.loc[:, 'RIDE1989':'RIDE2019']
df_pcr = df.loc[:, 'PCR1989':'PCR2019']

# Combine relevant data into a single DataFrame
df_combined = pd.concat([df_id, df_cycle, df_predictors, df_ratings], axis=1)
df_crk = df_combined.loc[:, crk_columns]
selected_rows = [] # Initialize a list to store rows that meet criteria

# Iterate through the DataFrame to apply selection criteria
for index, row in df_crk.iterrows():
    condition = []
    cyc_new_year = df.loc[index, 'cycnewyr']
    cyc_last_year = df.loc[index, 'cyclastyr']
    age_vector = []
    quality_vector = []
    i = 0

    for col in df_crk.columns:
        year = int(col[3:])
        col_name = col

        if (year >= cyc_new_year) and (year <= cyc_last_year):
            age_vector.append(i)
            i += 1
            quality = df_crk.loc[index, col]

            if pd.notna(quality):
                quality_vector.append(quality)
            else:
                condition.append(False)

        if len(quality_vector) < len(age_vector):
            condition.append(False)

        for n in range(1, len(quality_vector)):
            if quality_vector[n] > quality_vector[n - 1]:
                condition.append(False)

        if 0 in quality_vector:
            condition.append(False)

    result = all(condition)
    selected_rows.append(result)

df_selected = df_combined[selected_rows] # Create a new DataFrame with selected rows
predictor_columns = ['cycage', 'MAXSPEED', 'ESALS'] # Define predictor columns

# Filter rows with NaNs or 0s in predictor columns
df_selected = df_selected.dropna(subset=predictor_columns)
df_selected = df_selected[df_selected[predictor_columns].ne(0).all(axis=1)]

df_selected = df_selected.drop_duplicates() # Remove duplicate rows
# df_selected.to_csv("cleanedPredictorsAndRatings.csv", index=False) # Save the cleaned DataFrame to a CSV file

# Calculate and print the percentage of retained data
unclean_length = len(pd.read_csv(file_path))
clean_length = len(df_selected)
percentage_retained = (clean_length / unclean_length) * 100

print(f'Before the selection criteria were applied, there were {unclean_length} rows.')
print(f'After the selection criteria were applied, there are {clean_length} rows.')
print(f'That means that we retained {percentage_retained:.2f}% of the data.')

Before the selection criteria were applied, there were 14388 rows.
After the selection criteria were applied, there are 13625 rows.
That means that we retained 94.70% of the data.
